In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

In [ ]:
df= pd.read_csv("../input/rfcx-species-audio-detection/train_tp.csv")
df.head()

In [ ]:
df.describe()

In [ ]:
print(df.species_id.unique())

In [ ]:
 len(df) - len(df.recording_id.unique()) 

In [ ]:
fp = pd.read_csv("../input/rfcx-species-audio-detection/train_fp.csv")

In [ ]:
 len(fp) - len(fp.recording_id.unique()) 

In [ ]:
duplicate = [i  for i in fp.recording_id.unique() if i in df.recording_id.unique()]

In [ ]:
df[df['recording_id']=='015113cad']

In [ ]:
fp[fp['recording_id']=='015113cad']

In [ ]:
df.describe()

In [ ]:
df.t_min.hist()

In [ ]:
df.t_max.hist()

In [ ]:
df['duration'] = df['t_max'] - df['t_min']

In [ ]:
df.duration.hist()

In [ ]:
df[df['recording_id']=='c12e0a62b']

## Remove duplicate


In [ ]:
df_train = df.copy()
df_train.drop_duplicates(subset=['recording_id'], keep=False, inplace=True)

In [ ]:
df_train['type'] = df_train['species_id']
df_train['species_id'] = df_train['species_id'].apply(lambda x: [x])
df_train['t_min'] = df_train['t_min'].apply(lambda x: [x])
df_train['t_max'] = df_train['t_max'].apply(lambda x: [x])


In [ ]:
df_train.head()

In [ ]:
dt = df.recording_id.value_counts()
index = dt[dt>1].index

In [ ]:
recording_id_all = []
species_id_all = []
t_min_all = []
t_max_all = []
for idx in index:
    tmp = df[df['recording_id']==idx].values
    species_id_tmp = []
    t_min_tmp = []
    t_max_tmp = []
    for item in tmp:
        species_id_tmp.append(item[1])
        t_min_tmp.append(item[3])
        t_max_tmp.append(item[5])
    recording_id_all.append(idx)
    species_id_all.append(species_id_tmp)
    t_min_all.append(t_min_tmp)
    t_max_all.append(t_max_tmp)

In [ ]:
df_dup = pd.DataFrame({"recording_id": recording_id_all,
                     "species_id":species_id_all,
                      "t_min":t_min_all,
                      "t_max":t_max_all,
                      "type":25
                      })

In [ ]:
df_all = pd.concat((df_train, df_dup))

In [ ]:
df_all.sample(20)

In [ ]:
from sklearn.model_selection import StratifiedKFold

def sampling_k_elements(group, k=50):
    if len(group) < k:
        return group
    return group.sample(k)
    
train_full_df = (
            df_all.groupby("type")
            .apply(sampling_k_elements)
            .reset_index(drop=True)
        )

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=2020)

train_full_df["fold"] = -1
for fold_id, (train_index, val_index) in enumerate(skf.split(train_full_df, train_full_df["type"])):
    train_full_df.iloc[val_index, -1] = fold_id
train_full_df.head()

In [ ]:
for use_fold in range(5):
#     train_full_df = pd.read_csv("/content/drive/Shareddrives/Working/Kaggle/RainForest/train_tp.csv")
    train_file_list = train_full_df.query("fold != @use_fold")
    val_file_list = train_full_df.query("fold == @use_fold")
    print("[fold {}] train: {}, val: {}".format(use_fold, len(train_file_list), len(val_file_list)))
    print(train_file_list.type.value_counts())

In [ ]:
train_full_df.duration.hist()

In [ ]:
train_full_df.to_csv("train_preprocessing.csv",index=False)

In [ ]:
len(train_full_df)- len(train_full_df.recording_id.value_counts())

In [ ]:
for i in train_full_df.species_id.values:
    for j in i:
        pass

In [ ]:
dfx= pd.read_csv("train_preprocessing.csv")

In [ ]:
for i in dfx.species_id.values:
    for j in i:
        pass

In [ ]:
train_full_df.head()

## False positive

In [ ]:
fp = pd.read_csv("../input/rfcx-species-audio-detection/train_fp.csv")
fp.head()

In [ ]:
fp = fp[fp['recording_id'].isin(train_full_df.recording_id.tolist())]

In [ ]:
df_train = fp.copy()
df_train.drop_duplicates(subset=['recording_id'], keep=False, inplace=True)
df_train['type'] = df_train['species_id']
df_train['species_id'] = df_train['species_id'].apply(lambda x: [x])
df_train['t_min'] = df_train['t_min'].apply(lambda x: [x])
df_train['t_max'] = df_train['t_max'].apply(lambda x: [x])

In [ ]:
dt = fp.recording_id.value_counts()
index = dt[dt>1].index
recording_id_all = []
species_id_all = []
t_min_all = []
t_max_all = []
for idx in index:
    tmp = fp[fp['recording_id']==idx].values
    species_id_tmp = []
    t_min_tmp = []
    t_max_tmp = []
    for item in tmp:
        species_id_tmp.append(item[1])
        t_min_tmp.append(item[3])
        t_max_tmp.append(item[5])
    recording_id_all.append(idx)
    species_id_all.append(species_id_tmp)
    t_min_all.append(t_min_tmp)
    t_max_all.append(t_max_tmp)

In [ ]:
df_dup = pd.DataFrame({"recording_id": recording_id_all,
                     "species_id":species_id_all,
                      "t_min":t_min_all,
                      "t_max":t_max_all,
                      "type":25
                      })

df_all = pd.concat((df_train, df_dup))

In [ ]:
df_all

In [ ]:
df_all = df_all[['recording_id', 'species_id', 't_min', 't_max']]

In [ ]:
df_all.columns = ['recording_id', 'FPspecies_id', 'FPt_min', 'FPt_max']

In [ ]:
df_all.head()

In [ ]:
df_x = train_full_df.join(df_all.set_index('recording_id'), on='recording_id') 

In [ ]:
df_x

In [ ]:
df_x.to_csv("train_tp_fp.csv",index=False)